In [438]:
import pandas
import numpy as np
from operator import itemgetter

In [439]:
api_datafile_name = 'GOOGLE API BUSSINESS TYPE NAMES.xlsx'
target_data_file = 'armadale_cleaned.xlsx'

In [440]:
def get_google_api_data(api_datafile_name):
    google_api = pandas.read_excel(api_datafile_name,header=0,sheet_name = 'Only_matched')
    google_api['GOOGLE API ALL BUSINESS TYPE'] = google_api['GOOGLE API ALL BUSINESS TYPE'].str.strip()
    google_api['GOOGLE API ALL BUSINESS TYPE'] = google_api['GOOGLE API ALL BUSINESS TYPE'].str.lower()

    return google_api

In [441]:
def get_target_datafile(target_data_file):
    data= pandas.read_excel(target_data_file,header=0)
    data = pandas.DataFrame(data)
    return data
    

In [442]:
def get_target_business_types(data):
    business_type = data['temp_business_types'].str.split(',', expand=False)
    return business_type 

In [443]:
def classification_dictionarys(google_api):
    google_api['Weight'] = google_api['Weight'].astype(int)
    classification  = dict(zip(google_api['GOOGLE API ALL BUSINESS TYPE'], google_api['Classification']))
    Category  = dict(zip(google_api['GOOGLE API ALL BUSINESS TYPE'], google_api['Category']))
    Category_code  = dict(zip(google_api['GOOGLE API ALL BUSINESS TYPE'], google_api['Category code']))
    Sub_category  = dict(zip(google_api['GOOGLE API ALL BUSINESS TYPE'], google_api['Sub-category']))
    Sub_category_code  = dict(zip(google_api['GOOGLE API ALL BUSINESS TYPE'], google_api['Sub- category code']))

    classification  = {a : (c, b) for (a, b), c in zip(classification.items(), google_api['Weight'])} 
    Category  = {a : (c, b) for (a, b), c in zip(Category.items(),google_api['Weight'])} 
    Category_code  = {a : (c, b) for (a, b), c in zip(Category_code.items(),google_api['Weight'])} 
    Sub_category  = {a : (c, b) for (a, b), c in zip(Sub_category.items(), google_api['Weight'])} 
    Sub_category_code  = {a : (c, b)for (a, b), c in zip(Sub_category_code.items(), google_api['Weight'])} 
  
    return classification,Category,Category_code,Sub_category,Sub_category_code


In [444]:
classification,Category,Category_code,Sub_category,Sub_category_code = classification_dictionarys(google_api)
Category_code

{'airport': (2, 'A2.00'),
 'amusement_park': (3, 'F1.00'),
 'aquarium': (3, 'F1.00'),
 'art_gallery': (3, 'F1.00'),
 'bakery': (5, 'A7.00'),
 'bar': (5, 'B4.00'),
 'beauty_salon': (5, 'F2.00'),
 'bicycle_store': (3, 'A14.00'),
 'book_store': (3, 'A14.00'),
 'bowling_alley': (5, 'F2.00'),
 'bus_station': (2, 'A2.00'),
 'cafe': (5, 'B1.00'),
 'campground': (3, 'F2.00'),
 'casino': (3, nan),
 'church': (5, 'E8.00'),
 'city_hall': (2, 'A2.00'),
 'clothing_store': (3, 'A14.00'),
 'convenience_store': (5, 'A2.00'),
 'department_store': (3, 'A14.00'),
 'doctor': (5, 'E1.00'),
 'drugstore': (3, 'A14.00'),
 'electronics_store': (3, 'A14.00'),
 'embassy': (3, 'A2.00'),
 'fire_station': (3, 'A2.00'),
 'florist': (3, 'A14.00'),
 'funeral_home': (3, 'A14.00'),
 'furniture_store': (3, 'A14.00'),
 'gas_station': (5, 'A2.00'),
 'gym': (3, 'F2.00'),
 'hair_care': (3, 'F2.00'),
 'hardware_store': (3, 'A14.00'),
 'health': (3, nan),
 'hindu_temple': (5, 'E8.00'),
 'home_goods_store': (3, 'A14.00'),
 'hos

In [445]:
def class_matching(business_type_list,google_api) :

    classification,Category,Category_code,Sub_category,Sub_category_code = classification_dictionarys(google_api)
  
    classification_l = []
    Category_l = []
    Category_code_l = []
    Sub_category_l = []
    Sub_category_code_l = []

    for k in business_type_list :
        key = k.strip()
        classification_l.append(classification.get(key,(0,'undefined')))
        Category_l.append(Category.get(key.strip(),(0,'undefined')))
        Category_code_l.append(Category_code.get(key.strip(),(0,'undefined')))
        Sub_category_l.append(Sub_category.get(key.strip(),(0,'undefined')))
        Sub_category_code_l.append(Sub_category_code.get(key.strip(),(0,'undefined'))
) 

#remove na
    classification_l =[x for x in classification_l if x == x]
    Category_l = [x for x in  Category_l if x == x]
    Category_code_l = [x for x in Category_code_l if x == x]
    Sub_category_l = [x for x in Sub_category_l if x == x]
    Sub_category_code_l = [x for x in Sub_category_code_l if x == x]


    #get maximum
    classification_l =max(classification_l,key=lambda item:item[0], default=[(0,'undefined')])
    Category_l = max(Category_l,key=lambda item:item[0], default=[(0,'undefined')])
    Category_code_l =max(Category_code_l,key=lambda item:item[0], default=[(0,'undefined')])
    Sub_category_l = max(Sub_category_l,key=lambda item:item[0], default=[(0,'undefined')])
    Sub_category_code_l = max(Sub_category_code_l ,key=lambda item:item[0], default=[(0,'undefined')])

#remove duplicate
    classification_l =list(dict.fromkeys(classification_l))
    Category_l = list(dict.fromkeys(Category_l))
    Category_code_l = list(dict.fromkeys(Category_code_l))
    Sub_category_l = list(dict.fromkeys(Sub_category_l ))
    Sub_category_code_l = list(dict.fromkeys(Sub_category_code_l))
   
    result = {
     'classification':   [classification_l[1], 'code'],
     'category_one':     [Category_l[1], Category_code_l[1]],
     'sub_category_one': [Sub_category_l[1],Sub_category_code_l[1]]}
   

    return result

In [446]:
def matched_targeted_data(data,business_type,google_api):
    


    for i in range(len(data)):
        matched = class_matching(business_type[i],google_api)

        data['classification_name'][i]= matched['classification'][0]
        data['category_1'][i]= matched['category_one'][0]
        data['sub_category_1'][i]= matched['category_one'][1]
        data['category_2'][i]= matched['sub_category_one'][0]

    data.to_excel(r'Matched_Armadale_with_list.xlsx', index = False)

    return data



In [447]:
google_api = get_google_api_data(api_datafile_name)


targeted_data = get_target_datafile(target_data_file)
targeted_business_type =get_target_business_types(targeted_data)


#r = class_matching(targeted_business_type[1],google_api)
#r['classification']

matched_targeted_data(targeted_data,targeted_business_type,google_api)

/var/folders/1_/fcmjcggj0f78hwz7ngj0rxmc0000gn/T/ipykernel_1245/309553550.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['classification_name'][i]= matched['classification'][0]
/var/folders/1_/fcmjcggj0f78hwz7ngj0rxmc0000gn/T/ipykernel_1245/309553550.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['category_1'][i]= matched['category_one'][0]
/var/folders/1_/fcmjcggj0f78hwz7ngj0rxmc0000gn/T/ipykernel_1245/309553550.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

,local_government_area,business_id,places_id,collection_year,business_name,temp_business_types,classification_name,category_1,sub_category_1,category_2,...,original_lga_provided_address,formatted_address,contact_1,contact_2,email,website,"menu_(yes,_provided_on_website/no)",children's_menu_provided_(yes/no),opening_hours,notes
0,"Armadale, City of",NaN,ChIJoWnLRmuTMioRNsPZuUhB6ho,2022,Armadale Primary School Canteen,"primary_school, school, point_of_interest, est...",INSTITUTIONAL FOOD,Education,E6.00,Primary school,...,1 Carradine Road BEDFORDALE WA 6112,"1 Carradine Rd, Mount Nasura WA 6112, Australia",(08) 9391 4500,NaN,NaN,http://armadaleps.wa.edu.au,NaN,NaN,"[""Monday: 8:30 AM \u2013 2:45 PM"",""Tuesday: 8:...",NaN
1,"Armadale, City of",NaN,ChIJXw2JQxaTMioRTcu9c23ef70,2022,Armadale Senior High School Canteen,"secondary_school, school, point_of_interest, e...",INSTITUTIONAL FOOD,Education,E6.00,High school,...,169 South Western Highway ARMADALE WA 6112,"169 S Western Hwy, Armadale WA 6112, Australia",(08) 9497 6400,NaN,NaN,http://www.ashs.wa.edu.au/,NaN,NaN,"[""Monday: 8:00 AM \u2013 4:00 PM"",""Tuesday: 8:...",NaN
2,"Armadale, City of",NaN,ChIJeZMNGAOTMioRddFiyCjwpZE,2022,Australian Christian College (formerly Brookda...,"school, primary_school, university, point_of_i...",INSTITUTIONAL FOOD,Education,E6.00,Primary school,...,10 Ninth Road HILBERT WA 6112,"10 Ninth Rd, Brookdale WA 6112, Australia",(08) 9391 1500,NaN,NaN,https://www.acc.edu.au/darlingdowns/,NaN,NaN,"[""Monday: 8:00 AM \u2013 4:00 PM"",""Tuesday: 8:...",NaN
3,"Armadale, City of",NaN,ChIJN9znsoGTMioRdmDxoC8den0,2022,Cecil Andrews Senior High School Canteen,"school, point_of_interest, establishment",INSTITUTIONAL FOOD,Education,NaN,NaN,...,39 Seville Drive SEVILLE GROVE WA 6112,"39 Seville Dr, Seville Grove WA 6112, Australia",(08) 9234 3400,NaN,NaN,http://www.cecilandrewscollege.wa.edu.au/,NaN,NaN,"[""Monday: 8:30 AM \u2013 4:00 PM"",""Tuesday: 8:...",NaN
4,"Armadale, City of",NaN,ChIJCw4NLXyTMioRtk7LjOSi9co,2022,Challis Primary School P&C Canteen,"primary_school, school, point_of_interest, est...",INSTITUTIONAL FOOD,Education,E6.00,Primary school,...,40 Braemore Street ARMADALE WA 6112,"40 Braemore St, Armadale WA 6112, Australia",(08) 9391 4100,NaN,NaN,http://www.challiscommunityprimaryschool.wa.ed...,NaN,NaN,"[""Monday: Open 24 hours"",""Tuesday: Open 24 hou...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,"Armadale, City of",NaN,ChIJfw2lIb2UMioR0Ke5wCKys08,2022,River Road Lunchbar,"food, point_of_interest, store, establishment",FOOD RETAIL,NaN,NaN,NaN,...,43 River Road KELMSCOTT WA 6111,"42 River Rd, Kelmscott WA 6111, Australia",-,NaN,NaN,-,NaN,NaN,"[""Monday: 8:00 AM \u2013 7:00 PM"",""Tuesday: 8:...",NaN
485,"Armadale, City of",NaN,ChIJw-dl_YSUMioR-BdsdyC7B10,2022,Champion Lunch Bar,"restaurant, food, point_of_interest, establish...",FOOD SERVICE,Restaurant,NaN,NaN,...,1/1 Brant Road KELMSCOTT WA 6111,"Unit 1/1 Brant Rd, Kelmscott WA 6111, Australia",(08) 9390 0775,NaN,NaN,-,NaN,NaN,"[""Monday: 6:00 AM \u2013 2:30 PM"",""Tuesday: 6:...",NaN
486,"Armadale, City of",NaN,ChIJkYVD7XaTMioRuvJHcqSXXDY,2022,United in Diversity WA Inc.,premise,FOOD SERVICE,NaN,NaN,NaN,...,22A Terrigal Way ARMADALE WA 6112,"22A Terrigal Way, Armadale WA 6112, Australia",-,NaN,NaN,-,NaN,NaN,"""-""",NaN
487,"Armadale, City of",NaN,ChIJi-37-4DsMioRnka17iQ4S24,2022,Leah Anastasia Designs,premise,FOOD SERVICE,NaN,NaN,NaN,...,28A Hawkstone Road ROLEYSTONE WA 6111,"28 Hawkstone Rd, Roleystone WA 6111, Australia",-,NaN,NaN,-,NaN,NaN,"""-""",NaN


UNIT TEST

In [450]:
addr = class_matching(["cafe", "store", "food", "point_of_interest", "establishment"],google_api)
assert addr == {'classification': ['FOOD SERVICE', 'code'],
 'category_one': ['Café/coffee shop', 'B1.00'],
 'sub_category_one': [np.nan, np.nan]}



addr = class_matching(["tourist_attraction", "point_of_interest", "establishment"],google_api)
assert addr == {'classification': ['ACCOMMODATION/ RECREATION SERVICES', 'code'],
 'category_one': ['Entertainment venue', 'F1.00'],
 'sub_category_one': [np.nan, np.nan]}

In [451]:
class_matching(["tourist_attraction", "point_of_interest", "establishment"],google_api)

{'classification': ['ACCOMMODATION/ RECREATION SERVICES', 'code'],
 'category_one': ['Entertainment venue', 'F1.00'],
 'sub_category_one': [nan, nan]}